In [ ]:
print('sairam sairam')

sairam


In [7]:
import pandas as pd
import numpy as np
import os
print(os.getcwd())
os.chdir("../")
print(os.getcwd())

d:\AI\E2E-Wine-Quality-project\research
d:\AI\E2E-Wine-Quality-project


In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    

In [17]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml, create_directories

In [27]:
class ConfigurationManager:
    
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])    

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config
        

In [ ]:
import os
import urllib.request as request
import zipfilea
from ml_project import logger   
from ml_project.utils.common import get_size

In [32]:
class DataIngestion:
    def __init__(self,
                 config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url=self.config.source_url,
                                filename=self.config.local_data_file)
            logger.info(f"File {filename} downloaded ok with hearders: \n  {headers}")
        else:
            logger.info(f"File already exists : {get_size(self.config.local_data_file)}")
            
    def extract_zipfile(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [34]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()
    
except Exception as e:
    print(e)

[2025-03-01 17:53:04,649: INFO: Yaml file config\config.yaml loaded successfully]
[2025-03-01 17:53:04,652: INFO: Yaml file params.yaml loaded successfully]
[2025-03-01 17:53:04,656: INFO: Yaml file schema.yaml loaded successfully]
[2025-03-01 17:53:04,657: INFO: Created directory at artifacts]
[2025-03-01 17:53:04,660: INFO: Created directory at artifacts/data_ingestion]
[2025-03-01 17:53:06,248: INFO: File artifacts/data_ingestion/data.zip downloaded ok with hearders: 
  X-GUploader-UploadID: AHMx-iHPNNv3eqmKKaget8EmT_ooZp9seV8QHlH3_tCogURhoNG07ypO3eQ7rzMy-LdWW_igKiDbAGY
Expires: Sat, 01 Mar 2025 12:23:11 GMT
Date: Sat, 01 Mar 2025 12:23:11 GMT
Cache-Control: private, max-age=0
Last-Modified: Sat, 21 Sep 2019 19:06:16 GMT
ETag: "055e79eac2d90b1f7031be4d9d39d379"
x-goog-generation: 1569092776878130
x-goog-metageneration: 1
x-goog-stored-content-encoding: identity
x-goog-stored-content-length: 26176
Content-Type: application/zip
x-goog-hash: crc32c=HYQfcw==
x-goog-hash: md5=BV556sLZCx9